___
# FOOTBALL--DATA--EXTRACTOR
---
This script extracts and combines data from multiple CSV files
and saves it into the directory of the script.

Basic Football statistics for 11 countries, 17 leagues, 12 years
Data is freely available at www.Football-Data.co.uk

The script extracts only part of the available data points.

Check their notes:
https://www.football-data.co.uk/notes.txt
"""

# Data

Data source:
https://www.football-data.co.uk/

In [88]:
#################################
#-------------------------------#
#---FOOTBALL--DATA--EXTRACTOR---#
#-------------------------------#
#################################

# import shortuuid as uid
import datetime as dt
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm
import requests



example link to csv file:
https://www.football-data.co.uk/mmz4281/1011/E0.csv
                                      season/league

In the link, `2122` means season *2021/2022* and `E0.csv` means file name.
File naming:

- E0 : Premier league

- E1 : Championship

- E2 : League 1

- E3 : League 2

- etc.

In [89]:
#Codes and names for ALL LEAGUES
leagues = {
        'E0':['England', 'Premier League'],
        'E1':['England', 'Championship'],
        'SP1':['Spain', 'Primera Division'],
        'SP2':['Spain', 'Segunda Division'],
        'I1':['Italy', 'Serie A'],
        'I2':['Italy', 'Serie B'],
        'D1':['Germany', 'Bundesliga 1'],
        'D2':['Germany', 'Bundesliga 2'],
        'F1':['France', 'Le Championnat'],
        'F2':['France', 'Division 2'],
        'SC0':['Scotland','Premier League'],
        'SC1':['Scotland','Division 1'],
        'N1':['Netherlands','Eredivisie'],
        'P1':['Portugal','Liga I'],
        'T1':['Turkey','Futbol Ligi 1'],
        'G1':['Greece','Ethniki Katigoria'],
        'B1':['Belgium','Jupiler League']}

In [26]:
# #Generating the links
# links = []
# for div in wanted_divisions:

#   for season in seasons:
#     # print(f'https://www.football-data.co.uk/mmz4281/{season}/{div}.csv')
#     links.append(f'https://www.football-data.co.uk/mmz4281/{season}/{div}.csv')
# print('Links generated')

Links generated


In [90]:
#list of wanted columns from every file
wanted = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'Referee',
          'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']

base_attributes = ['id','Country', 'League', 'Div', 'Season', 'Date', 'HomeTeam', 'AwayTeam', 'Referee',
                   'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']

data = pd.DataFrame(columns=base_attributes)   #creating empty table with the main attributes
print('Table template created.')
data

Table template created.


,id,Country,League,Div,Season,Date,HomeTeam,AwayTeam,Referee,FTHG,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR


In [91]:
def file_download(url, path):
    response = requests.get(url)

    if response.status_code == 200:
        with open(path, 'wb') as file:
            file.write(response.content)
        # print(f"File downloaded successfully and saved at {path}")
    else:
        print(f"Failed to download {path}")

# Example Usage
# url = "https://www.football-data.co.uk/mmz4281/1112/E0.csv"
# path = "E0.csv"
# file_download(url, save_path)


In [92]:
start = 2012                   #starting year
now = dt.date.today().year     #current year
ly = int(str(start)[2:])       #equals to the last 2 digits of start year
years = []                     #will be populated with lists like the example

for x in range(1, now-start+1):
    season = start + x
    seasons = [str(ly)+str(ly+1), str(season-1)+"-"+str(season)]
    ly += 1
    years.append(seasons)

#getting the data
links = []

for league in tqdm(leagues.keys(), desc=f"Extracting data"):
    for i in tqdm(years, desc=f'{leagues[league][0]} - {leagues[league][1]}'):
        year = i[0]
        links.append(f'https://www.football-data.co.uk/mmz4281/{year}/{league}.csv')
        name = league+"-"+str(season)+".csv"
        # print(name)
        # print(links[-1])
        file_download(links[-1], name)
        temp = pd.read_csv(name, encoding='windows-1252')
        # print(f'{year}/{league}')
        try:
            temp = temp[wanted]
        except:
            for col in wanted:
                if col not in temp.columns:
                    temp[col] = np.nan

        temp['Country'] = leagues[league][0]
        temp['League'] = leagues[league][1]
        temp['Season'] = i[1]
        temp['id'] = 0
        temp = temp[base_attributes]
        data = pd.concat([data, temp],ignore_index=True)
        # print(f'adding {temp.shape[0]} rows to the table')

print("Data extraction completed")
#creating ids for all matches
# data['id'] = [uid.uuid() for x in data['id']]

#exporting data into the location of the script
data.to_excel(f'Euro-Football_{start}-{now}.xlsx', sheet_name= 'Euro_Football', index=False)
data.to_csv(f'Euro-Football_{start}-{now}.csv', index=False)

print(f"Data downloaded and saved here ---> '{os.getcwd()}' \n")
data.info()

Extracting data:   0%|          | 0/17 [00:00<?, ?it/s]

England - Premier League:   0%|          | 0/11 [00:00<?, ?it/s]

England - Championship:   0%|          | 0/11 [00:00<?, ?it/s]

Spain - Primera Division:   0%|          | 0/11 [00:00<?, ?it/s]

Spain - Segunda Division:   0%|          | 0/11 [00:00<?, ?it/s]

...

Portugal - Liga I:   0%|          | 0/11 [00:00<?, ?it/s]

Turkey - Futbol Ligi 1:   0%|          | 0/11 [00:00<?, ?it/s]

Greece - Ethniki Katigoria:   0%|          | 0/11 [00:00<?, ?it/s]

Belgium - Jupiler League:   0%|          | 0/11 [00:00<?, ?it/s]

Data extraction completed
Data downloaded and saved here ---> '/content' 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63272 entries, 0 to 63271
Data columns (total 27 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        63272 non-null  object
 1   Country   63272 non-null  object
 2   League    63272 non-null  object
 3   Div       63167 non-null  object
 4   Season    63272 non-null  object
 5   Date      63167 non-null  object
 6   HomeTeam  63167 non-null  object
 7   AwayTeam  63167 non-null  object
 8   Referee   12922 non-null  object
 9   FTHG      63165 non-null  object
 10  FTAG      63165 non-null  object
 11  FTR       63165 non-null  object
 12  HTHG      63076 non-null  object
 13  HTAG      63076 non-null  object
 14  HTR       63076 non-null  object
 15  HS        47282 non-null  object
 16  AS        47282 non-null  object
 17  HST       47282 non-null  object
 18  AST       47282 non-null  object
 19  HF        459

In [118]:
data['Date'] = pd.to_datetime(data['Date'])

#creating year column
data['Year'] = data['Date'].dt.year

data = data[['id', 'Country', 'League', 'Div', 'Season', 'Date', 'Year', 'HomeTeam', 'AwayTeam', 'Referee',
             'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR',
             'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']]

data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63272 entries, 0 to 63271
Data columns (total 28 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        63272 non-null  object        
 1   Country   63272 non-null  category      
 2   League    63272 non-null  category      
 3   Div       63167 non-null  category      
 4   Season    63272 non-null  category      
 5   Date      63167 non-null  datetime64[ns]
 6   Year      63167 non-null  float64       
 7   HomeTeam  63167 non-null  category      
 8   AwayTeam  63167 non-null  category      
 9   Referee   12922 non-null  object        
 10  FTHG      63165 non-null  object        
 11  FTAG      63165 non-null  object        
 12  FTR       63165 non-null  category      
 13  HTHG      63076 non-null  object        
 14  HTAG      63076 non-null  object        
 15  HTR       63076 non-null  category      
 16  HS        47282 non-null  object        
 17  AS        47